In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# введите имя папки на гугл диске, в которую вы сохранили папку vision, содержащую .py файлы, а также папки datatasets и classifiers
# например, cv/assignments/assignment1/vision/
FOLDERNAME = None

assert FOLDERNAME is not None "[!] Enter the foldername."

%cd drive/MyDrive
%cp -r $FOLDERNAME ../../
%cd ../../
%cd cs231n/datasets/
!bash get_datasets.sh
%cd ../../

# k-Nearest Neighbor (kNN)

*Выполните задания и сдайте их, сохранив полученные результаты, вместе со всем необходимым кодом, включая код за пределами этого ноутбука, например, код в папке `vision`*

kNN работает в две стадии:

- Во время обучения классификатор просто запоминает обучающую выборку
- Во время тестирования классификатор тестирует каждое изображение тестовой выборки, равнивая его с каждым изображением обучающей выборки и присваивая тестовому изображению класс, который чаще всего встречается среди k самых похожих изображений
- k - это гиперпараметр, который находится с помощью кросс-валидации

В этом задании вам нужно будет реализовать перечисленные шаги и изучить процесс классификации изображения, кросс-вализацию и на практике обедиться в эффективности выполнения векторынх операций.

In [ ]:
# Этот код нужно запустить для предватирельных настроек

import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# Чтобы графики выводились в том же окне, а не новом
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # размер изображений
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Загрузка выборки CIFAR-10.
cifar10_dir = 'vision/datasets/cifar-10-batches-py'

# Для того, чтобы данные не дублировались при повторной загрузке (например, если вы запускаете эту ячейку, не обновив сессию), очистим переменные
try:
   del X_train, y_train
   del X_test, y_test
   print('Загруженны данны удален')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# ДУдим Убедимся, что данные загружены и выведем размеры обучающих и тестовых данных# Убедимся, что данные загружены и выведем размеры обучающих и тестовых данных ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
# Выведем несколько примеров из выборки
# Покажем несколько изображений каждого класса
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Выберем подмножество данных для того, чтобы обучение работало быстрее (меньше данных - меньше считать, но результат может оказаться хуже)
num_training = 5000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 500
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

# Трансформируем изображения в векторы
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

In [ ]:
from cs231n.classifiers import KNearestNeighbor

# Создание экземпляра классификатора kNN
# И обучение (которое в нашем случае сводится к запоминанию экземпляров обучающей выборки)

Мы будем классифицировать тестовые данные с помощью классификатора kNN. Напомню, что процесс классификации состоит из двух этапов:

1. Мы должны посчитать расстояние между всем тестовыми изображениями и всеми обучающими изображениями.
2. Имея эти расстояния, для каждого тестового изображения необходимо найти k ближайших изобржаений обучающией выборки и проанализировать классы этих соседей для каждого тестового изображения.

Мы начнем с вычисления матрицы расстояний между всем обучающими и тестовыми изображениями. Например, если у нас есть **Ntr** изображений обучающей выборки и **Nte** изображений, в результате мы должны получить матрицу **Nte x Ntr**, в которой каждоый элемент (i,j) - это расстояние между тестовым изображением i и обучающим изображением j.

**Примечание: для вычисления расстояний в данном задании нельзя использовать функцию np.linalg.norm() из numpy.**

Откройте `vision/classifiers/k_nearest_neighbor.py` и реализуйте функцию `compute_distances_two_loops` которая использует вложенные циклы (правда, очень неэффективные) для построения матрицы расстояний междоу всеми изображениями тестовой выборки и всеми изображениями обучающей выборки.

In [ ]:
# Откройте cs231n/classifiers/k_nearest_neighbor.py и реализуйте
# compute_distances_two_loops

# Протестируйте свою реализацию:
dists = classifier.compute_distances_two_loops(X_test)
print(dists.shape)

In [ ]:
# Матриц расстояний можно визуализировать: каждая строка - это тестовое изображение,
# показывающая расстояние между текущим тестовым изображением и всем обучающими изображениями
plt.imshow(dists, interpolation='none')
plt.show()

**Вопрос** 

Матрица расстояний имеет закономерности: некоторые строки или колонки выглядят ярче остальных (при этом черный цвет говорит о небольшом расстоянии между изображениями, а белый - о большом).

- Какие изображения и почему двают яркие строки?
- Какие изображения и почему дают яркие колонки?

$\color{blue}{\textit Ответ:}$ *Дайте ответ здесь.*



In [ ]:
# Теперь реализуйте функцию predict_labels и запустите код ниже:
# k = 1 (самый ближайшй сосед).
y_test_pred = classifier.predict_labels(dists, k=1)

# Вычислите и выведете долю правильно классифицированных изображений
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

Точность должна равняться примерно `27%`. Теперь попробуем взять большее значение `k`, например `k = 5`:

In [ ]:
y_test_pred = classifier.predict_labels(dists, k=5)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

Точность распознавания должна быть чуть-чуть выше `k = 1`.

In [ ]:
# Теперь попытаемся ускорить вычисление матрицы расстояний частично используя векторные
# операции, оставив только один цикл. реализуйте функцию compute_distances_one_loop
# и запустите код ниже:
dists_one = classifier.compute_distances_one_loop(X_test)

# Чтобы убедиться в правильности векторной реализации, убедимся, что она совпадает
# с нашей обычной реализацией. Для этого есть несколько способов, мы будем использовать
# норму Фробениуса - квадратный корень из суммы квадратов разностей двух матриц,
# вычесленных поэлементно. По-другому ее можено представить так: матрицы транформируются
# в два вектора, и находится Евклидово расстояние между векторами
difference = np.linalg.norm(dists - dists_one, ord='fro')
print('One loop difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [ ]:
# Теперь реализуйте полностью векторизированную версию в функции
# compute_distances_no_loops и запустите код ниже:
dists_two = classifier.compute_distances_no_loops(X_test)

# проверим, что матрица расстояний совпадает с матрицей, полученной первым методом:
difference = np.linalg.norm(dists - dists_two, ord='fro')
print('No loop difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [ ]:
# Сравним производительность наших реализаций
def time_function(f, *args):
    """
    Call a function f with args and return the time (in seconds) that it took to execute.
    """
    import time
    tic = time.time()
    f(*args)
    toc = time.time()
    return toc - tic

two_loop_time = time_function(classifier.compute_distances_two_loops, X_test)
print('Two loop version took %f seconds' % two_loop_time)

one_loop_time = time_function(classifier.compute_distances_one_loop, X_test)
print('One loop version took %f seconds' % one_loop_time)

no_loop_time = time_function(classifier.compute_distances_no_loops, X_test)
print('No loop version took %f seconds' % no_loop_time)

# NOTE: в зависимости от испоьзуемого компьютера, вы можете не увидеть разницы в ускорении
# при переходе от реализации с одним циклом к реализации без циклов, иногда вы можете даже
# наблюдать падение производительности

### Кросс-валидация (кросс-проверка)

Мы реализовании классификатор, но произвольным образом задали `k = 5`. Ниже вам будет необходимо подобрать значение `k` с помощью кросс-валидации.

In [ ]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []
################################################################################
# TODO:                                                                        #
# Разделите обучающие данные на части. после этого, X_train_folds и            #
# y_train_folds должны быть списками длины num_folds, где y_train_folds[i] -   #
# это вектор меток классов для изображений X_train_folds[i].                   #
# Подсказка: используйте функцию numpy.array_split                             #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Словарь, содержащий точность классификации для различных значений k, которые мы
# находим в процессе кросс-валидации. После запуска кросс-валидации 
# k_to_accuracies[k] должен быть списком длины num_folds, состоящий из различных
# значений точности, которые мы получаем применяя данное k.
k_to_accuracies = {}


################################################################################
# TODO:                                                                        #
# Выполните кросс-валидацию по для того, чтобы найти наилучшее значение k. Для # 
#  каждого возможного значения k, запустите классификатор num_folds, где в     #
# каждом случае оставляйте одну часть для проверки точности, остальные - для   # 
# обучения. Сохраните точность распознавания для каждой проверочной части в    # 
# словаре k_to_accuracies.                                                     #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Выведите вычесленные точности
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

In [ ]:
# Выведите график, показывающий результаты экчперимента
for k in k_choices:
    accuracies = k_to_accuracies[k]
    plt.scatter([k] * len(accuracies), accuracies)

# Выведите линию тренда со "свечами", отражающими среднеквадратичное отклонение
accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
plt.errorbar(k_choices, accuracies_mean, yerr=accuracies_std)
plt.title('Cross-validation on k')
plt.xlabel('k')
plt.ylabel('Cross-validation accuracy')
plt.show()

In [ ]:
# На основе результатов кросс-валидации выберите лучшее значение k, обучите
# классифиатор снова на всех обучающих данных и проверьте точность на 
# тестовой выборке. Вы должны получить около 28% точности
best_k = 1

classifier = KNearestNeighbor()
classifier.train(X_train, y_train)
y_test_pred = classifier.predict(X_test, k=best_k)

# Вычислите и выведите точность
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))